# Synthesize a coloring book

In [1]:
import numpy as np
import skimage
from PIL import Image
from PIL.Image import Image as PilImage

from diffusers import StableDiffusionPipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoImageProcessor, Swin2SRForImageSuperResolution
import torch

import openai

# !pip install python-docx 
from docx import Document
from docx.shared import Inches

2023-04-28 08:35:08.113042: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# openAI key
openai.api_key = ""
# for azure openAI
# your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_base =  "" 
openai.api_version = '' # this may change in the future
openai.api_type = 'azure'
#This will correspond to the custom name you chose for your deployment when you deployed a model. 
deployment_name='' 

def load_prompter():
    prompter_model = AutoModelForCausalLM.from_pretrained("microsoft/Promptist")
    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"
    return prompter_model, tokenizer

prompter_model, prompter_tokenizer = load_prompter()

def load_upscaler():
    processor = AutoImageProcessor.from_pretrained("caidas/swin2SR-classical-sr-x2-64")
    model = Swin2SRForImageSuperResolution.from_pretrained("caidas/swin2SR-classical-sr-x2-64")
    return model, processor
    
upscale_model, upscale_processor = load_upscaler()

## Inputs

In [5]:
topic = "zoo animals playing sports"
number_pages = 40
images_to_generate = 5

## Load models

In [6]:
model_id = "dreamlike-art/dreamlike-photoreal-2.0"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

/home/titan/anaconda3/lib/python3.9/site-packages/safetensors/torch.py:98: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(filename, framework="pt", device=device) as f:
/home/titan/anaconda3/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/home/titan/anaconda3/lib/python3.9/site-packages/torch/storage.py:899: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be t

## Helper functions

In [7]:
# create a sketch for coloring based in an input image
def generate_coloring_page(input: PilImage) -> PilImage:
    # Convert to grayscale if needed
    if input.mode != "L":
        input = input.convert("L")
    np_image = np.asarray(input)
    
    # detect edges
    np_image = skimage.filters.sobel(np_image)
    # convert to 8 bpp
    np_image = skimage.util.img_as_ubyte(np_image)
    # Invert to get dark edges on a light background
    np_image = 255 - np_image
    # Improve the contrast
    np_image = skimage.exposure.rescale_intensity(np_image)
    
    return Image.fromarray(np_image)

# Create ideas for each desired page of the coloring book
def generate_coloring_book_page_descriptions(topic, number_pages):
    prompt = "Create "+str(number_pages)+" ideas for a coloring book on the topic of "+topic+"/n/nIdeas for coloring book topic:"
    response = openai.Completion.create(
            engine=deployment_name,
            prompt=prompt,
            temperature=0.7,
            max_tokens=1500
        )
    result = response.choices[0].text.strip()
    return result, prompt

# Create fun facts about the image
def generate_fun_facts(topic):
    prompt = "Create fun facts about "+topic+"/n/nFun facts:"
    response = openai.Completion.create(
            engine=deployment_name,
            prompt=prompt,
            temperature=0.7,
            max_tokens=1500
        )
    result = response.choices[0].text.strip()
    return result, prompt

# parse a string to list based on return characters
def parse_string(input_string):
    result = []
    lines = input_string.split('\n')
    for line in lines:
        if 'playing' in line:
            result.append(line.strip())
    return result

def generate(plain_text):
    input_ids = prompter_tokenizer(plain_text.strip()+" Rephrase:", return_tensors="pt").input_ids
    eos_id = prompter_tokenizer.eos_token_id
    outputs = prompter_model.generate(input_ids, do_sample=False, max_new_tokens=75, num_beams=8, num_return_sequences=8, eos_token_id=eos_id, pad_token_id=eos_id, length_penalty=-1.0)
    output_texts = prompter_tokenizer.batch_decode(outputs, skip_special_tokens=True)
    res = output_texts[0].replace(plain_text+" Rephrase:", "").strip()
    return res

def enhance_image(image):
    # prepare image for the model
    inputs = upscale_processor(image, return_tensors="pt")

    # forward pass
    with torch.no_grad():
        outputs = upscale_model(**inputs)

    # postprocess
    output = outputs.reconstruction.data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.moveaxis(output, source=0, destination=-1)
    output = (output * 255.0).round().astype(np.uint8)  # float32 to uint8
    
    return Image.fromarray(output)

## Workflow

In [8]:
# Create page ideas given the desired topic
page_ideas = generate_coloring_book_page_descriptions(topic, number_pages)
print(page_ideas)

('Zoo Animals Playing Sports\n\n1. Gorilla playing basketball\n2. Elephant playing soccer\n3. Zebra playing baseball\n4. Tiger playing golf\n5. Giraffe playing tennis\n6. Kangaroo playing hockey\n7. Koala playing cricket\n8. Chimpanzee playing volleyball\n9. Ostrich playing badminton\n10. Lemur playing rugby\n11. Hippo playing cricket\n12. Warthog playing lacrosse\n13. Llama playing football\n14. Sloth playing softball\n15. Gorilla playing bowling\n16. Elephant playing golf\n17. Zebra playing Frisbee\n18. Tiger playing soccer\n19. Giraffe playing rugby\n20. Kangaroo playing dodgeball\n21. Koala playing basketball\n22. Chimpanzee playing hockey\n23. Ostrich playing baseball\n24. Lemur playing badminton\n25. Hippo playing tennis\n26. Warthog playing croquet\n27. Llama playing volleyball\n28. Sloth playing Frisbee\n29. Gorilla playing tennis\n30. Elephant playing softball\n31. Zebra playing cricket\n32. Tiger playing dodgeball\n33. Giraffe playing badminton\n34. Kangaroo playing rugby\n35

In [9]:
# parse the page ideas into a list for each page
pages = parse_string(page_ideas[0])
print(pages)

['1. Gorilla playing basketball', '2. Elephant playing soccer', '3. Zebra playing baseball', '4. Tiger playing golf', '5. Giraffe playing tennis', '6. Kangaroo playing hockey', '7. Koala playing cricket', '8. Chimpanzee playing volleyball', '9. Ostrich playing badminton', '10. Lemur playing rugby', '11. Hippo playing cricket', '12. Warthog playing lacrosse', '13. Llama playing football', '14. Sloth playing softball', '15. Gorilla playing bowling', '16. Elephant playing golf', '17. Zebra playing Frisbee', '18. Tiger playing soccer', '19. Giraffe playing rugby', '20. Kangaroo playing dodgeball', '21. Koala playing basketball', '22. Chimpanzee playing hockey', '23. Ostrich playing baseball', '24. Lemur playing badminton', '25. Hippo playing tennis', '26. Warthog playing croquet', '27. Llama playing volleyball', '28. Sloth playing Frisbee', '29. Gorilla playing tennis', '30. Elephant playing softball', '31. Zebra playing cricket', '32. Tiger playing dodgeball', '33. Giraffe playing badmint

In [ ]:
# create fun facts and images for each page
fun_facts = []
images = []
count = 0
# iterate for each desired page
for page in pages:
    prompt = "photo, "+page
    # improve the image prompt with a model
    new_prompt = generate(prompt)
    # iterate for the desired number of images to choose from for each page
    for image_num in range(0, images_to_generate):
        # create an image based on the improved prompt
        image = pipe(new_prompt).images[0]
        # enhance/upscale the generated image
        enhanced_image = enhance_image(image)
        # create a coloring book image based on the generated image
        coloring_book_image = generate_coloring_page(enhanced_image)
        # save the image with the format image_pageNumber_imageNumber as jpg
        coloring_book_image.save("image_"+str(count)+"_"+str(image_num)+".jpg")
    # add the last image generated for each page as a placeholder for the generated word document
    images.append(enhanced_image)
    # add the fun facts for each page in a list
    fun_facts.append(generate_fun_facts(page))
    count = count + 1

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# write to a word document
document = Document()
document.add_heading('Coloring Book', 0)

p = document.add_paragraph('A coloring book about '+topic)
#p.add_run('bold').bold = True
#p.add_run(' and some ')
#p.add_run('italic.').italic = True

count = 0
for page in pages:
    document.add_page_break()
    document.add_heading('Heading, level 1', level=1)
    document.add_paragraph('Fun facts about '+pages[count], style='Intense Quote')
    document.add_picture("image_"+str(count)+".jpg", width=Inches(5.0))
    document.add_paragraph(fun_facts[count])
#document.add_paragraph(
#    'first item in ordered list', style='List Number'
#)



#records = (
#    (3, '101', 'Spam'),
#    (7, '422', 'Eggs'),
#    (4, '631', 'Spam, spam, eggs, and spam')
#)

#table = document.add_table(rows=1, cols=3)
#hdr_cells = table.rows[0].cells
#hdr_cells[0].text = 'Qty'
#hdr_cells[1].text = 'Id'
#hdr_cells[2].text = 'Desc'
#for qty, id, desc in records:
#    row_cells = table.add_row().cells
#    row_cells[0].text = str(qty)
#    row_cells[1].text = id
#    row_cells[2].text = desc

document.save('demo.docx')